<a href="https://colab.research.google.com/github/jogden33/HousingProject/blob/master/Housing_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Pandas and do some data exploration on the three datasets

In [ ]:
import pandas as pd
import re as re
import numpy as np
import random

# Read in our 3 datasets
Housing = pd.read_csv("/housing-info.csv")
Income = pd.read_csv("/income-info.csv")
County = pd.read_csv("/zip-city-county-state.csv")

In [ ]:
# Find every instance of corrupt data and replace it with NaN
Housing.replace(to_replace = r'^[A-Z]{4}$', value=np.nan, regex=True, inplace=True)

# Delete the rows where guid is NaN
Housing.dropna(axis = 0, subset = ['guid'], inplace = True)
Housing.head(25)

In [ ]:
# Replace missing values for several of the columns
Housing.loc[Housing['housing_median_age'].isnull(), 'housing_median_age'] = Housing['housing_median_age'].apply(lambda v: np.random.randint(10,50))
Housing.loc[Housing['total_rooms'].isnull(), 'total_rooms'] = Housing['total_rooms'].apply(lambda v: np.random.randint(1000, 2000))
Housing.loc[Housing['total_bedrooms'].isnull(), 'total_bedrooms'] = Housing['total_bedrooms'].apply(lambda v: np.random.randint(1000,2000))
Housing.loc[Housing['population'].isnull(), 'population'] = Housing['population'].apply(lambda v: np.random.randint(5000, 10000))
Housing.loc[Housing['households'].isnull(), 'households'] = Housing['households'].apply(lambda v: np.random.randint(500, 2500))
Housing.loc[Housing['median_house_value'].isnull(), 'median_house_value'] = Housing['median_house_value'].apply(lambda v: np.random.randint(100000, 250000))
Housing.head(25)


In [ ]:
# Create a dataset that strips down non-missing values for zip, city, state, and country. This gives us a reference table to allow for imputation of missing
# ZIP codes in our mail file.
CountyImputed = County[['zip_code', 'city', 'state', 'county']].copy()
CountyImputed.replace(to_replace = r'^[A-Z]{4}$', value=np.nan, regex=True, inplace=True)
CountyImputed.dropna(axis = 0, subset = ['zip_code'], inplace = True)
CountyImputed.sort_values(by=['state','city','county'], inplace = True)
CountyImputed['NewZip']=CountyImputed.zip_code.str[:1]+'0000'
CountyImputed.drop_duplicates(['state','city','county'], keep='last', inplace = True)
CountyImputed.tail(25)

In [ ]:
Income.head(25)

In [ ]:
# Clean the income file 
Income.replace(to_replace = r'^[A-Z]{4}$', value=np.nan, regex=True, inplace=True)
Income.dropna(axis = 0, subset = ['guid'], inplace = True)
Income.loc[Income['median_income'].isnull(), 'median_income'] = Income['median_income'].apply(lambda v: np.random.randint(100000, 750000))
Income.head(25)

In [ ]:
#County.replace(to_replace = r'^[A-Z]{4}$', value=np.nan, regex=True, inplace=True)
#County.dropna(axis = 0, subset = ['guid'], inplace = True)
#County.head(25)

In [ ]:
# Begin merging our datasets
Housing_Zip = Housing.merge(County, how = 'outer', on = ['guid', 'zip_code'])
Housing_Zip.head(25)

In [ ]:
Housing_Zip_Income = Housing_Zip.merge(Income, how = 'outer', on = ['guid', 'zip_code'])
Housing_Zip_Income.head(25)

In [ ]:
# Step 4: Create a reference dataset with non-duplicate zip codes so we can impute missing Zip Codes
ZipTable = County[['zip_code', 'city', 'state', 'county']].copy()

# Next, replace all of the corrupted zip codes with NaN (missing value) and drop all of those rows
ZipTable.replace(to_replace = r'^[A-Z]{4}$', value=np.nan, regex=True, inplace=True)
ZipTable.dropna(axis = 0, subset = ['zip_code'], inplace = True)

# Finally, sort the data by state, city, county, in that order and create a new variable that strings the first
# number from the zip code to '0000'. Also, get rid of any duplicates.
ZipTable.sort_values(by=['state','city','county'], inplace = True)
ZipTable['NewZip']=ZipTable.zip_code.str[:1]+'0000'
ZipTable.drop_duplicates(['state','city','county'], keep='last', inplace = True)

# Now we have a clean reference dataset from which we can join our other two datasets to obtain zip codes for missing values
Housing_Zip_Income2 = Housing_Zip_Income.merge(ZipTable, how = 'left', on = ['state', 'city', 'county'])
Housing_Zip_Income2.tail(50)

In [ ]:
Housing_Zip_Income2.loc[Housing_Zip_Income2['zip_code_x'].isnull(), 'zip_code_x'] = Housing_Zip_Income2["NewZip"]
Housing_Zip_Income2['id'] = range(1,1+len(Housing_Zip_Income2))
Housing_Zip_Income2.rename(columns = {'zip_code_x':'zip_code', 'housing_median_age':'median_age'}, inplace = True)
HousingData = Housing_Zip_Income2[['id','guid','zip_code', 'city','state','county','median_age','total_rooms','total_bedrooms','population','households','median_income','median_house_value']].copy()
HousingData.head(50)